In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 86.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.5 MB/s eta 0:00:00


In [3]:
import pandas as pd

#Get data
df = pd.read_csv('./drive/MyDrive/output.csv', sep=',')
df = df.dropna().reset_index()

#Select part of data we want to keep
df = df[['text','summary']]

#Clean text
df['text'] = df['text'].apply(lambda x: x.replace('\n',' '))
df['summary'] = df['summary'].apply(lambda x: x.replace('\n',' '))

#Select only part of it (makes testing faster)
writeups = df
writeups.head()

,text,summary
0,"When you visit the website, you get redirected...",Visit website and notice the `/?file=wc.php` r...
1,"Description: ""You can steal a car if you steal...",Examine the website source to find routes `/lo...
2,"Description: ""This is my file library. I don't...",Analyze the JavaScript code of the Express app...
3,"Description: ""People who get violent get that ...",Google the challenge description and discover ...
4,"Description: ""My nephew is a fussy eater and i...",Change the cookie value to the base64 value of...


In [4]:
from transformers import BartForConditionalGeneration, BartTokenizer, BartConfig

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

In [ ]:
import numpy as np

# Extract the text from the random index
#random_index = np.random.randint(0, len(writeups))
#sample_writeup = writeups.loc[random_index, 'text']
#matching_rows = writeups[writeups['text'].str.contains('more_table')]
#matching_index = matching_rows.index[0]
#sample_writeup = writeups.loc[matching_index, 'text']
#inputs = tokenizer([sample_writeup], return_tensors='pt')

sample_text = """
When you examine the source code, you'll notice the application uses `bodyParser.urlencoded` with `extended: true`. This allows arrays and objects to be passed in the request body. In the `/login` POST route, the output has not been stringified, so it is possible to pass an object in the query statement.
[JavaScript code: Configure bodyParser and create SQL query]

The exploit is to pass an object as the password parameter, with a known attribute. This causes the SQL query to be evaluated in a way that allows logging in without knowing the correct password. We'll refer to the official `mysql` docs for more information on how objects are converted into comma-separated attributes in SQL queries.

The final payload for this exploit is:
[HTTP POST request: Send payload with object as password]

From here, you can just take the cookie you received, and use that to visit `/flag`. Alternatively, you can use a Python script to automate the process:
[Python script: Send payload, get the flag, and print it]

You can run this script and use `grep` to find the flag.
"""
inputs = tokenizer([sample_text], return_tensors='pt')
summary_ids = model.generate(inputs['input_ids'], max_length=500, early_stopping=False)

summary_ids

print([tokenizer.decode(g, skip_special_tokens=True) for g in summary_ids])

['The exploit is to pass an object as the password parameter, with a known attribute. This causes the query to be evaluated in a way that allows logging in without knowing the correct password. You can run this script and use `grep` to find the flag. Alternatively, you can use a Python script to automate the process.']
